In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras import layers
from keras.layers import Activation, Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, Dense, BatchNormalization, Flatten, Input, Add
from keras.models import Model, load_model
from keras.utils import to_categorical

Using TensorFlow backend.
C:\Users\Asus\anaconda3\envs\MyEnv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Asus\anaconda3\envs\MyEnv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Asus\anaconda3\envs\MyEnv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Asus\anaconda3\envs\MyEnv\lib\site-packag


## Identity Block

The identity block is used when the input and output have the same dimensions
<br>The identity block has 3 parts

1.  1. 2D Covolution : f1  `(1, 1)` filters with `valid` padding and stride `(1, 1)` 
    2. Batch Normalization
    3. ReLU Non-Linearity 
    
    <br>
    
2.  1. 2D Covolution : f2  `(f, f)` filters with `same` padding and stride `(1, 1)` 
    2. Batch Normalization
    3. ReLU Non-Linearity
    
    <br>
    
3.  1. 2D Convolution : f3  `(1, 1)` filters with `valid` padding and stride `(1, 1)` 
    2. Batch Normalization 

In [2]:
def identity_block(X,  f, filters):
    """
    Identity Block Implementation
    
    Arguements:
    X - input images of shape (N, H, W, C)
    f - Shape of filter in the path
    filters - list of integers giving the number of filters in different components
    """
    
    
    #Shape of filters
    f1, f2, f3 = filters
    
    #Input value for the shortcut path
    X_add = X
    
    
    #First component of the identity block
    X = Conv2D(filters=f1, kernel_size=(1, 1), padding = 'valid', name = '')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    #Second component of the identity block
    X = Conv2D(filters = f2, kernel_size = (f, f), padding = 'same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    #Last compenent of the identity block
    X = Conv2D(filters = f3, kernel_size = (1, 1), padding = 'valid')(X)
    X = BatchNormalization()(X)
    
    # Adding the shortcut to the output
    X = Add()([X, X_add])
    
    return X

## Convolutional Block
The convolutional block is used when the input and output dimensions are not the same, so as to adjust the size of the input to match with the output
<br> 
1.  1. 2D Covolution : f1  `(1, 1)` filters with `valid` padding and stride `(1, 1)`
    2. Batch Normalization
    3. ReLU Non-Linearity 
    
    <br>
    
2.  1. 2D Covolution : f2  `(f, f)` filters with `same` padding and stride `(1, 1)`
    2. Batch Normalization
    3. ReLU Non-Linearity
    
    <br>
    
3.  1. 2D Convolution : f3  `(1, 1)` filters with `valid` padding and stride `(1, 1)` 
    2. Batch Normalization
    
    <br>
For the shortcut path :-
1.  1. 2D Convolution : f3  `(1, 1)` filters with `valid` padding and stride `(1, 1)` 
    2. Batch Normalization 

In [3]:
def convolutional_block(X,  f, filters, s = 2):
    """
    Convolutional Block Implementation
    
    Arguements:
    X - input images of shape (N, H, W, C)
    f - Shape of filter in the path
    filters - list of integers giving the number of filters in different components
    s - strides
    """
    
    #Shape of filters
    f1, f2, f3 = filters
    
    #Input value for the shortcut path
    X_add = X
    
    
    #First component of the identity block
    X = Conv2D(filters=f1, kernel_size=(1, 1), padding = 'valid', strides = (s, s))(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    #Second component of the identity block
    X = Conv2D(filters = f2, kernel_size = (f, f), padding = 'same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    #Last compenent of the identity block
    X = Conv2D(filters = f3, kernel_size = (1, 1), padding = 'valid')(X)
    X = BatchNormalization()(X)
    
    #Shortcut Path
    X_add = Conv2D(filters = f3, kernel_size = (1, 1), padding = 'valid', strides = (s, s))(X_add)
    X_add = BatchNormalization()(X_add)
    
    X = Add()([X_add, X])
    X = Activation('relu')(X)
    
    return X

## Building the ResNet Model

In [4]:
def ResNet(input_shape = (32, 32, 3), num_classes = 10):
    """
    The main ResNet Model
    
    Arguements:
    input_shape : input shape of the image
    num_classes : Total number of classes for classification
    """
    #Defining the Input
    X_input = Input(input_shape)
    
    X = Conv2D(filters = 32, kernel_size = (3, 3))(X_input)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), strides = (1, 1))(X)
    
    #Using the identity and convolutional blocks
    X = convolutional_block(X, 5, [64, 64, 128])
    X = identity_block(X, 3, [64, 64, 128])
    X = identity_block(X, 3, [64, 64, 128])
    
    X = convolutional_block(X, 3, [128, 128, 256])
    X = identity_block(X, 3, [128, 128, 256])
    X = identity_block(X, 3, [128, 128, 256])
    
    X = convolutional_block(X, 5, [256, 256, 512])
    X = identity_block(X, 3, [256, 256, 512])
    X = identity_block(X, 3, [256, 256, 512])
    
    X = convolutional_block(X, 3, [512, 512, 1024])
    X = identity_block(X, 3, [512, 512, 1024])
    X = identity_block(X, 3, [512, 512, 1024])
    
    
    X = Flatten()(X)
    output = Dense(num_classes, activation='softmax')(X)

    #Model Initialization
    model = Model(inputs = X_input, outputs = output, name = "ResNet")
    
    return model
    

In [5]:
model = ResNet(input_shape = (32, 32, 3), num_classes=10)

In [6]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [7]:
model.summary()

Model: "ResNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 30, 30, 32)   896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 30, 30, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 30, 30, 32)   0           batch_normalization_1[0][0]      
_____________________________________________________________________________________________

## Data Preprocessing

In [8]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [9]:
def process_image(image):
    R = image[:1024]
    G = image[1024:2048]
    B = image[2048:]
    
    r = R.reshape((32, 32))
    g = G.reshape((32, 32))
    b = B.reshape((32, 32))
    
    img = np.dstack((r, g, b))
    return img

In [10]:
def final_data(dataset):
    images = []
    for image in dataset:
        img = process_image(image)
        images.append(img)
    images = np.array(images)
    return images

## Training model on data

In [11]:
for i in range(5):
        batch_name = 'cifar-10-batches-py/data_batch_' + str(i+1)
        data_batch_i = unpickle(batch_name)
        data_ = data_batch_i[b'data']
        labels = data_batch_i[b'labels']
        
        x_batch = final_data(data_)
        y_batch = labels
        y_batch = to_categorical(y_batch, num_classes = 10)
        
        model.fit(x_batch, y_batch, epochs = 20, batch_size = 32)


Epoch 1/20
10000/10000 [==============================] - 40s 4ms/step - loss: 3.7368 - accuracy: 0.2708
Epoch 2/20
10000/10000 [==============================] - 31s 3ms/step - loss: 2.2514 - accuracy: 0.3755
Epoch 3/20
10000/10000 [==============================] - 31s 3ms/step - loss: 1.7005 - accuracy: 0.4464
Epoch 4/20
10000/10000 [==============================] - 31s 3ms/step - loss: 1.4381 - accuracy: 0.4940
Epoch 5/20
10000/10000 [==============================] - 30s 3ms/step - loss: 1.3083 - accuracy: 0.5429
Epoch 6/20
10000/10000 [==============================] - 32s 3ms/step - loss: 1.1156 - accuracy: 0.6082
Epoch 7/20
10000/10000 [==============================] - 30s 3ms/step - loss: 0.9761 - accuracy: 0.6541
Epoch 8/20
10000/10000 [==============================] - 29s 3ms/step - loss: 0.8457 - accuracy: 0.7036 1s - loss: 0
Epoch 9/20
10000/10000 [==============================] - 31s 3ms/step - loss: 0.7063 - accuracy: 0.7522
Epoch 10/20
10000/10000 [================

10000/10000 [==============================] - 41s 4ms/step - loss: 0.0641 - accuracy: 0.9787
Epoch 16/20
10000/10000 [==============================] - 38s 4ms/step - loss: 0.1608 - accuracy: 0.9596
Epoch 17/20
10000/10000 [==============================] - 36s 4ms/step - loss: 0.0787 - accuracy: 0.9754
Epoch 18/20
10000/10000 [==============================] - 37s 4ms/step - loss: 0.0434 - accuracy: 0.9875
Epoch 19/20
10000/10000 [==============================] - 35s 3ms/step - loss: 0.0325 - accuracy: 0.9908
Epoch 20/20
10000/10000 [==============================] - 37s 4ms/step - loss: 0.0223 - accuracy: 0.9929
Epoch 1/20
10000/10000 [==============================] - 43s 4ms/step - loss: 0.9109 - accuracy: 0.7181
Epoch 2/20
10000/10000 [==============================] - 41s 4ms/step - loss: 0.3904 - accuracy: 0.8690
Epoch 3/20
10000/10000 [==============================] - 40s 4ms/step - loss: 0.1594 - accuracy: 0.9487
Epoch 4/20
10000/10000 [==============================] - 41s

## Test batch

In [12]:
test_batch = unpickle('cifar-10-batches-py/test_batch')

In [13]:
y = test_batch[b'labels']

In [14]:
y_test = to_categorical(y, num_classes = 10)

In [15]:
X = test_batch[b'data']

In [16]:
X_test = final_data(X)

In [17]:
X_test.shape, y_test.shape

((10000, 32, 32, 3), (10000, 10))

In [18]:
hist = model.evaluate(X_test, y_test)

10000/10000 [==============================] - 8s 767us/step


In [19]:
hist

[1.7036216952323913, 0.7251999974250793]